In [ ]:
import outlines
from enum import Enum
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModelForCausalLM
from typing import List

# ---------------------------------------------------------
# 1. SETUP: DEFINING THE "SHAPE" OF THE DATA
# ---------------------------------------------------------
# We aren't just writing a Python class here. Outlines will look at this 
# Pydantic model and convert it into a complex "Grammar" or Regex 
# behind the scenes. This acts like a stencil for the AI.
class TicketPriority(str, Enum):
    low = "low"
    medium = "medium"
    high = "high"
    urgent = "urgent"

class ServiceTicket(BaseModel):
    priority: TicketPriority
    category: str
    requires_manager: bool
    summary: str
    action_items: List[str]

# ---------------------------------------------------------
# 2. SETUP: LOADING THE BRAINS
# ---------------------------------------------------------
MODEL_NAME = "microsoft/Phi-3-mini-4k-instruct"

# We load the standard HuggingFace model. 
# 'trust_remote_code' is needed because Phi-3 uses custom architecture code.
# 'device_map="auto"' puts the model on your GPU if available.
llm = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, 
    device_map="auto", 
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

# THE MAGIC LINE:
# outlines.from_transformers wraps the standard LLM. 
# It intercepts the model's generation process. When the model tries to 
# pick the next word, Outlines checks the Pydantic schema (ServiceTicket)
# and bans any token that doesn't fit the JSON structure.
model = outlines.from_transformers(llm, tokenizer)

# ---------------------------------------------------------
# 3. THE INPUT
# ---------------------------------------------------------
customer_email = """
Subject: URGENT - Cannot access my account after payment
I paid for the premium plan 3 hours ago and still can't access any features.
Please fix this immediately or refund my payment.
"""

# We format the prompt using Phi-3's specific chat template tags
# (<|im_start|>, etc).
prompt = f"""<|im_start|>user
Analyze this customer email:
{customer_email}
<|im_end|>
<|im_start|>assistant
"""

# ---------------------------------------------------------
# 4. EXECUTION: GENERATING THE TICKET
# ---------------------------------------------------------
# Here is the fix for the confusing variable usage.
# When we pass `ServiceTicket` as the second argument, Outlines does three things:
# 1. Forces the LLM to output valid JSON matching the ServiceTicket schema.
# 2. Waits for generation to finish.
# 3. AUTOMATICALLY converts that JSON string into a Python 'ServiceTicket' object.
structured_ticket_object = model(
    prompt,
    ServiceTicket,
    max_new_tokens=500
)

# ---------------------------------------------------------
# 5. LOGIC: USING THE RESULT
# ---------------------------------------------------------
# Because 'structured_ticket_object' is already a Python object, 
# we can access properties using dot notation (.priority) immediately.
# No JSON parsing is required here.

def alert_manager(ticket):
    print(f"ALARM: {ticket.priority.upper()} priority issue: {ticket.summary}")

# We check the Enums and Booleans directly
if structured_ticket_object.priority == TicketPriority.urgent or structured_ticket_object.requires_manager:
    alert_manager(structured_ticket_object)
else:
    print("Ticket routed to standard queue.")